In [ ]:
import rasterio
import numpy as np
from tqdm import tqdm

# Open the TIFF file with 13 bands
with rasterio.open(r'F:\sentinel2.tif') as src:
    # Read the image data as a numpy array
    data = src.read()  # Read all bands
    crs = src.crs
    transform = src.transform

# Define the region size (90x90)
region_size = 128

# Create a progress bar for the outer loop
outer_progress = tqdm(total=(data.shape[1] // region_size) * (data.shape[2] // region_size), desc="Searching Regions")

# List to store the coordinates of found regions
found_regions = []

# Initialize the starting coordinates and counter
x, y, counter = 0, 0, 1

# Iterate through the image to find suitable regions
while x < data.shape[1]:
    # Update the outer progress bar
    outer_progress.update(1)

    while y < data.shape[2]:
        # Ensure the region does not go beyond the image dimensions
        if x + region_size > data.shape[1] or y + region_size > data.shape[2]:
            break

        region = data[:, x:x+region_size, y:y+region_size]  # Extract all bands for the region

        # Calculate the transform for the new region based on the original file
        window = rasterio.windows.Window(y, x, region_size, region_size)
        new_transform = src.window_transform(window)

        # Save the region as a new properly georeferenced TIFF file
        output_filename = fr'F:\{counter:05d}.tif'
        with rasterio.open(output_filename, 'w', driver='GTiff', height=region.shape[1], width=region.shape[2], count=region.shape[0], 
                            dtype=region.dtype, crs=crs, transform=new_transform) as dst:
            dst.write(region)

        # Store the coordinates of the found region
        found_regions.append((x, y))

        y += region_size  # Move right by the region size
        counter += 1

    # Reset y for the next row
    y = 0
    x += region_size  # Move down by the region size

# Close the outer progress bar
outer_progress.close()

# Print the coordinates of the found regions
print("Coordinates of Found Regions:")
for x, y in found_regions:
    print(f"x={x}, y={y}")





